In [ ]:
import untangle
import pandas as pd

# drug types to include
drug_types = ["biotech", "small molecule"]
# list of unwanted drugs
unwanted_drugs = ["vaccine", "antibody", "toxin", "biological", "biopolymer", "oligonucleotide", "oligosaccharide"]

input_file = "C://Users/Peng/Downloads/full database.xml"
output_file = "C://Users/Peng/Downloads/peng_ddi.csv"

obj = untangle.parse(input_file)
columns = ["drugbank_id1", "drug1", "drugbank_id2", "drug2", "interaction"]
rows = []

for drug in obj.drugbank.drug:
    if drug["type"] not in drug_types:
        continue

    # Get primary drug id
    for id in drug.drugbank_id:
        if id["primary"] == "true":
            primary_id = id.cdata

    # Skip drugs without calculated properties or without drug interactions
    if not hasattr(drug, "drug_interactions") or len(drug.drug_interactions.cdata) == 0:
        continue

    # Check if drug name is in unwanted_drugs list
    drug_name = drug.name.cdata.lower()
    if any(unwanted_drug in drug_name for unwanted_drug in unwanted_drugs):
        continue

    for interaction in drug.drug_interactions.drug_interaction:
        # Check if interaction drug name is in unwanted_drugs list
        interaction_name = interaction.name.cdata.lower()
        if any(unwanted_drug in interaction_name for unwanted_drug in unwanted_drugs):
            continue
        
        rows.append([primary_id, drug.name.cdata, interaction.drugbank_id.cdata, interaction.name.cdata, interaction.description.cdata])

df = pd.DataFrame(rows, columns=columns)

# Write to CSV
with open(output_file, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(columns)
    writer.writerows(rows)
